## 1. Tokenización del texto

### Cargamos el texto 

In [2]:
trs = r'transcription.txt'
with open(trs, 'r') as f:
    text = f.read()

### Importamos OpenAI

In [32]:
import tiktoken
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
apikey = os.getenv("clave")
client = OpenAI(api_key=apikey)

### Número de tokens del texto

Queremos saber cuántos tokens tiene el texto. Para ello utilizamos una función creada por nosotros mismos y por otro lado usamos el propio contador de tokens de OpenAI. 

- Función *num_tokens_from_messages*

Dado un modelo de gpt y una lista de mensajes, medimos la lungitud, en tokens, de cada uno de los mensajes y sumamos el total. En este caso particular, nuestro texto transcrito es el único mensaje y tiene 3817 tokens. Cabe señalar que os tokens no tienen por qué coincidir con el número de palabras; una misma palabra puede contecer más de un token (prefijos, sufijos, desinencias, conjugaciones, etc.) y los signos de puntuación también figuran como tokens.



In [33]:
def num_tokens_from_messages(messages, model):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for message in messages:
        num_tokens += len(encoding.encode(message))
    return num_tokens

model = "gpt-3.5-turbo"

print(model)
# example token count from the function defined above
print(f"{num_tokens_from_messages([text], model)} prompt tokens counted by num_tokens_from_messages().")
# example token count from the OpenAI API
response = client.chat.completions.create(model=model,
                                          messages=[{"role": "user", "content": text}],
                                          temperature=0,
                                          max_tokens=1)
print(f'{response.usage.prompt_tokens} prompt tokens counted by the OpenAI API.')
print()

gpt-3.5-turbo
3825 prompt tokens counted by num_tokens_from_messages().
3832 prompt tokens counted by the OpenAI API.



## 2. Uso de gpt-4o con el texto

Le pedimos al modelo gpt-4o que formule 10 preguntas con sus correspondientes respuestas.

In [34]:
def generar_preguntas_respuestas(texto):
    preguntas_respuestas = []
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "assistant", "content": f"Genera 10 preguntas sobre el siguiente texto{texto}"}],
        temperature=0.6,
        max_tokens=200,
        top_p=0.7,
        frequency_penalty=0,
        presence_penalty=0
    )

    preguntas = completion.choices[0].message.content.split("\n")

    #generar respuestas por cada pregunta
    for pregunta in preguntas:
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "user", "content": "Responde cada pregunta con la mayor similitud al texto recibido, manteniendo la personalidad de origen, palabras claves, tono, forma de expresión y contexto."},
                {
                    "role": "user",
                    "content": pregunta
                }
            ],
            temperature=1.4,
            max_tokens=200,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )

        respuesta = completion.choices[0].message.content.split("\n")
        preguntas_respuestas.append((pregunta, respuesta))

    return preguntas_respuestas


generar_preguntas_respuestas(text)



[('1. ¿Qué diferencia notable se observa entre las fotografías de los años 50 y las imágenes modernas de la misma región del cielo?',
  ['En las fotografías de los años 50 de las regiones del cielo se observa notablemente menos claridad y nivel de detalle en comparación con las imágenes modernas. Esto se debe a los avances tecnológicos significativos en los equipos de observación astronómica, como la mejora en la resolución de los telescopios y las habilidades de procesamiento de imagen. Mientras las fotos antiguas ofrecían un esquema más básico y limitada capacidad para captar la tenue expresión de estas luces estelares, las modernas capturan un universo repleto de colores vibrantes y detalles intrigantes que antes eran invisibles, aportando una percepción mucho más vívida del firmamento que nos rodea.']),
 ('2. ¿Cuántas estrellas han sido reportadas como desaparecidas desde los años 50 según los científicos?',
  ['Hasta la fecha de corte de mi conocimiento, se han reportado cerca de 